In [1]:
!git clone https://github.com/Youbadawy/FinRL-Library.git

fatal: destination path 'FinRL-Library' already exists and is not an empty directory.


In [2]:
%cd FinRL-Library/
!pip install -r requirements.txt

/content/FinRL-Library


In [3]:
!pip install -U ipython
# !pip install colorama

Requirement already up-to-date: ipython in /usr/local/lib/python3.6/dist-packages (7.16.1)


In [4]:
%cd FinRL-Library/

[Errno 2] No such file or directory: 'FinRL-Library/'
/content/FinRL-Library


In [5]:
import nest_asyncio
nest_asyncio.apply()

In [6]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

%matplotlib inline
from finrl.config import config
from finrl.marketdata.yahoodownloader import YahooDownloader
from finrl.preprocessing.preprocessors import FeatureEngineer
from finrl.preprocessing.data import data_split
from finrl.env.env_stocktrading import StockTradingEnv
from finrl.model.models import DRLAgent
from finrl.trade.backtest import backtest_stats, backtest_plot, get_daily_return, get_baseline

from pprint import pprint

import sys
sys.path.append("../FinRL-Library")

import itertools

/usr/local/lib/python3.6/dist-packages/pyfolio/pos.py:28: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  ' to position notionals.'


In [7]:
import os
if not os.path.exists("./" + config.DATA_SAVE_DIR):
    os.makedirs("./" + config.DATA_SAVE_DIR)
if not os.path.exists("./" + config.TRAINED_MODEL_DIR):
    os.makedirs("./" + config.TRAINED_MODEL_DIR)
if not os.path.exists("./" + config.TENSORBOARD_LOG_DIR):
    os.makedirs("./" + config.TENSORBOARD_LOG_DIR)
if not os.path.exists("./" + config.RESULTS_DIR):
    os.makedirs("./" + config.RESULTS_DIR)

In [8]:
from finrl.config.configuration import Configuration
from finrl.config.directory_operations import create_userdata_dir
from finrl.commands import start_download_cryptodata, start_download_stockdata

In [9]:
#### CREATE USER DATA DIRECTORY IN DESIGNATED PATH, IF NO NAME INDICATED DEFAULT TO user_data
####### create dir to false if only to check existence of directory
create_userdata_dir("./user_data",create_dir=True)


# ###### Pull Configuration File (using finrl/config/configuration.py)
config = Configuration.from_files(["./notebooks/config.json"])

In [10]:
from finrl.config import setup_utils_configuration
from finrl.state import RunMode

ARGS_DOWNLOAD_DATA = {'config': ['./notebooks/config.json'], 'datadir': None, 
                      'user_data_dir': None, 'pairs': None, 'pairs_file': None, 
                      'days': 365, 'timerange': None, 
                      'download_trades': False, 'exchange': 'yahoo', 
                      'timeframes': '5m', 'erase': False, 
                      'dataformat_ohlcv': None, 'dataformat_trades': None}

### Adds to config ARGS stats for further use through config

config = setup_utils_configuration(ARGS_DOWNLOAD_DATA, RunMode.UTIL_EXCHANGE)
print(config.get("timeframes"))

5m


In [11]:
from finrl.tools.coin_search import *

tickers = ["FUBO", "JMIA", "AMZN", "APPN", "LUV", "IBIO", "BB","TSLA", "BABA", "GOOG", "RLLCF", "NIO", "RIOT", "BNGO", "MVIS", "IDEX", "NOK", "PLTR", "TSNP", "CCIV", "JNJ", "JPM", "MA", "PG", "BAC", "PFE", "UPS", "WFC", "SNOW", "USB", "INTC", "EA", "AMD", "NDA", "PYPL"  ]
len(tickers)

stocks_to_json("./notebooks/config.json", tickers)
config = Configuration.from_files(["./notebooks/config.json"])
print(config)

{'max_open_trades': 10, 'stake_currency': 'BTC', 'stake_amount': 0.1, 'tradable_balance_ratio': 0.99, 'fiat_display_currency': 'USD', 'timeframe': '1d', 'dry_run': True, 'exchange': {'name': 'yahoo', 'key': '', 'secret': '', 'ccxt_config': {'enableRateLimit': True}, 'ccxt_async_config': {'enableRateLimit': True, 'rateLimit': 200}, 'pair_whitelist': ['MBL/BNB', 'SC/BNB', 'WIN/BNB', 'IOST/BNB', 'BTT/BNB', 'HOT/BNB', 'COS/BNB', 'STMX/BNB', 'BNBDOWN/USDT', 'RSR/BNB', 'TROY/BNB', 'VET/BNB', 'MITH/BNB', 'ONE/BNB', 'VTHO/BNB', 'TRX/BNB', 'ANKR/BNB', 'MFT/BNB', 'SAND/BNB', 'IQ/BNB', 'WRX/BNB', 'XLM/BNB', 'CELR/BNB', 'FET/BNB', 'IOTA/BNB', 'ZIL/BNB', 'OGN/BNB', 'HBAR/BNB', 'COTI/BNB', 'STX/BNB', 'PERL/BNB', 'FTT/BNB', 'ONT/BNB', 'FIO/BNB', 'HARD/BNB', 'XRP/BNB', 'DIA/BNB', 'RVN/BNB', 'CTSI/BNB', 'SWRV/BNB', 'ARPA/BNB', 'CTK/BNB', 'CHZ/BNB', 'KAVA/BNB', 'ANT/BNB', 'DGB/BNB', 'FTM/BNB', 'JST/BNB', 'NEAR/BNB', 'MATIC/BNB'], 'pair_blacklist': []}, 'ticker_list': ['FUBO', 'JMIA', 'AMZN', 'APPN', 'LU

In [12]:
ARGS_DOWNLOAD_DATA = {'config': ['./notebooks/config.json'], 'datadir': None, 
                      'user_data_dir': None, 'days': None, 'timerange': "20020101-20210101",
                      'timeframes': ['1d'], 'erase': False}


start_download_stockdata(ARGS_DOWNLOAD_DATA)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [13]:
from finrl.data.fetchdata import FetchData
import pandas as pd
from finrl.config import TimeRange

In [14]:
df = FetchData(config).fetch_data_stock()
print(df.head)

Stock FUBO from ./user_data/data/yahoo/FUBO.json Fetched successfully...
Stock JMIA from ./user_data/data/yahoo/JMIA.json Fetched successfully...
Stock AMZN from ./user_data/data/yahoo/AMZN.json Fetched successfully...
Stock APPN from ./user_data/data/yahoo/APPN.json Fetched successfully...
Stock LUV from ./user_data/data/yahoo/LUV.json Fetched successfully...
Stock IBIO from ./user_data/data/yahoo/IBIO.json Fetched successfully...
Stock BB from ./user_data/data/yahoo/BB.json Fetched successfully...
Stock TSLA from ./user_data/data/yahoo/TSLA.json Fetched successfully...
Stock BABA from ./user_data/data/yahoo/BABA.json Fetched successfully...
Stock GOOG from ./user_data/data/yahoo/GOOG.json Fetched successfully...
Stock RLLCF from ./user_data/data/yahoo/RLLCF.json Fetched successfully...
Stock NIO from ./user_data/data/yahoo/NIO.json Fetched successfully...
Stock RIOT from ./user_data/data/yahoo/RIOT.json Fetched successfully...
Stock BNGO from ./user_data/data/yahoo/BNGO.json Fetched 

In [15]:
df.head()

,date,open,high,low,close,volume,tic,day
0,2002-01-02,16.280001,16.410000,15.950,16.389999,4475400,AMD,2
1,2002-01-02,10.930000,11.000000,10.480,10.960000,6519600,AMZN,2
2,2002-01-02,31.155001,31.549999,31.055,19.802853,8669200,BAC,2
3,2002-01-02,3.955000,4.098333,3.915,4.083333,5870400,BB,2
4,2002-01-02,30.245001,30.370001,29.805,30.019999,5836200,EA,2


In [16]:
df.sort_values(['date','tic'],ignore_index=True).head()

fe = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = config["TECHNICAL_INDICATORS_LIST"],
                    use_turbulence=False,
                    user_defined_feature = False)

processed = fe.preprocess_data(df)
processed.columns

Successfully added technical indicators


Index(['date', 'open', 'high', 'low', 'close', 'volume', 'tic', 'day', 'macd',
       'boll_ub', 'boll_lb', 'rsi_30', 'cci_30', 'dx_30', 'close_30_sma',
       'close_60_sma'],
      dtype='object')

In [17]:
print(processed.head())
processed.shape

         date       open       high  ...  dx_30  close_30_sma  close_60_sma
0  2002-01-02  16.280001  16.410000  ...  100.0     16.389999     16.389999
1  2002-01-02  10.930000  11.000000  ...  100.0     17.880000     17.880000
2  2002-01-02  31.155001  31.549999  ...  100.0     18.586667     18.586667
3  2002-01-02   3.955000   4.098333  ...  100.0     18.935000     18.935000
4  2002-01-02  30.245001  30.370001  ...  100.0     19.128000     19.128000

[5 rows x 16 columns]


(107764, 16)

In [18]:
print(processed.tail())
processed.shape

              date        open        high  ...  dx_30  close_30_sma  close_60_sma
107759  2020-12-31  699.989990  718.719971  ...  100.0      0.000455      0.000455
107760  2020-12-31    0.173000    0.173900  ...  100.0      0.000458      0.000458
107761  2020-12-31  166.789993  168.479996  ...  100.0      0.000462      0.000462
107762  2020-12-31   46.130001   46.660000  ...  100.0      0.000464      0.000464
107763  2020-12-31   29.750000   30.240000  ...  100.0      0.000467      0.000467

[5 rows x 16 columns]


(107764, 16)

In [24]:
list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max(), freq="1d").astype(str))
combination = list(itertools.product(list_date,list_ticker))
list_date

['2002-01-02',
 '2002-01-03',
 '2002-01-04',
 '2002-01-05',
 '2002-01-06',
 '2002-01-07',
 '2002-01-08',
 '2002-01-09',
 '2002-01-10',
 '2002-01-11',
 '2002-01-12',
 '2002-01-13',
 '2002-01-14',
 '2002-01-15',
 '2002-01-16',
 '2002-01-17',
 '2002-01-18',
 '2002-01-19',
 '2002-01-20',
 '2002-01-21',
 '2002-01-22',
 '2002-01-23',
 '2002-01-24',
 '2002-01-25',
 '2002-01-26',
 '2002-01-27',
 '2002-01-28',
 '2002-01-29',
 '2002-01-30',
 '2002-01-31',
 '2002-02-01',
 '2002-02-02',
 '2002-02-03',
 '2002-02-04',
 '2002-02-05',
 '2002-02-06',
 '2002-02-07',
 '2002-02-08',
 '2002-02-09',
 '2002-02-10',
 '2002-02-11',
 '2002-02-12',
 '2002-02-13',
 '2002-02-14',
 '2002-02-15',
 '2002-02-16',
 '2002-02-17',
 '2002-02-18',
 '2002-02-19',
 '2002-02-20',
 '2002-02-21',
 '2002-02-22',
 '2002-02-23',
 '2002-02-24',
 '2002-02-25',
 '2002-02-26',
 '2002-02-27',
 '2002-02-28',
 '2002-03-01',
 '2002-03-02',
 '2002-03-03',
 '2002-03-04',
 '2002-03-05',
 '2002-03-06',
 '2002-03-07',
 '2002-03-08',
 '2002-03-

In [25]:
processed["date"] = processed["date"].astype(str)
processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.fillna(0)
processed_full.sort_values(['date','tic'],ignore_index=True).head(10)

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma
0,2002-01-02,AMD,16.280001,16.410000,15.950,16.389999,4475400.0,2.0,0.000000,22.094359,13.665642,100.000000,66.666667,100.0,16.389999,16.389999
1,2002-01-02,AMZN,10.930000,11.000000,10.480,10.960000,6519600.0,2.0,0.066859,22.094359,13.665642,100.000000,66.666667,100.0,17.880000,17.880000
2,2002-01-02,APPN,0.000000,0.000000,0.000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
3,2002-01-02,BABA,0.000000,0.000000,0.000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
4,2002-01-02,BAC,31.155001,31.549999,31.055,19.802853,8669200.0,2.0,0.104728,22.443212,14.730121,100.000000,80.580876,100.0,18.586667,18.586667
5,2002-01-02,BB,3.955000,4.098333,3.915,4.083333,5870400.0,2.0,0.118663,22.378352,15.491648,99.414103,73.717080,100.0,18.935000,18.935000
6,2002-01-02,BNGO,0.000000,0.000000,0.000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
7,2002-01-02,CCIV,0.000000,0.000000,0.000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
8,2002-01-02,EA,30.245001,30.370001,29.805,30.019999,5836200.0,2.0,0.120553,22.232429,16.023570,97.061009,54.372966,100.0,19.128000,19.128000
9,2002-01-02,FUBO,0.000000,0.000000,0.000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000


In [26]:
processed_full.head(5)

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma
0,2002-01-02,AMD,16.280001,16.410000,15.950,16.389999,4475400.0,2.0,0.000000,22.094359,13.665642,100.000000,66.666667,100.0,16.389999,16.389999
1,2002-01-02,AMZN,10.930000,11.000000,10.480,10.960000,6519600.0,2.0,0.066859,22.094359,13.665642,100.000000,66.666667,100.0,17.880000,17.880000
2,2002-01-02,BAC,31.155001,31.549999,31.055,19.802853,8669200.0,2.0,0.104728,22.443212,14.730121,100.000000,80.580876,100.0,18.586667,18.586667
3,2002-01-02,BB,3.955000,4.098333,3.915,4.083333,5870400.0,2.0,0.118663,22.378352,15.491648,99.414103,73.717080,100.0,18.935000,18.935000
4,2002-01-02,EA,30.245001,30.370001,29.805,30.019999,5836200.0,2.0,0.120553,22.232429,16.023570,97.061009,54.372966,100.0,19.128000,19.128000


In [27]:
processed_full.tail()

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma
242860,2020-12-31,JMIA,42.500000,42.599998,39.835999,40.349998,8581900.0,3.0,2.259679e+00,30.177584,23.323416,63.813126,69.394656,43.245496,24.270333,17.431833
242861,2020-12-31,SNOW,299.700012,301.561005,280.730011,281.399994,5313800.0,3.0,-5.673874e-06,0.000563,0.000357,39.972449,-83.333333,100.000000,0.000460,0.000460
242862,2020-12-31,CCIV,10.070000,10.070000,10.010000,10.010000,1266200.0,3.0,2.972614e+00,32.168111,16.800889,65.547008,67.076192,40.751631,20.918667,15.987037
242863,2020-12-31,PLTR,24.629999,24.719999,23.520000,23.549999,39922500.0,3.0,-2.301788e-07,0.000565,0.000402,50.847458,200.000000,100.000000,0.000483,0.000483
242864,2020-12-31,RLLCF,0.000400,0.000500,0.000400,0.000500,8751800.0,3.0,-2.966120e-06,0.000567,0.000367,51.296129,12.500000,100.000000,0.000467,0.000467


In [29]:
#### DATA SPLIT TRAIN TRADE RATIO 70/30

trade_len = int(len(processed_full)*0.7)
train_len = int(len(processed_full)*0.3+1)
total = trade_len+train_len
print(f'trade length:{trade_len}, train length: {train_len}, for total len of {total} of {len(processed_full)}')

trade length:117207, train length: 50233, for total len of 167440 of 167440


In [30]:
processed_full.date.factorize()[0]

array([   0,    0,    0, ..., 4783, 4783, 4783])

In [31]:
train = data_split(processed_full, processed_full.date.min(),processed_full.date.loc[trade_len])
trade = data_split(processed_full, processed_full.date.loc[trade_len],processed_full.date.max())
print(len(train))
print(len(trade))

80815
86590


In [32]:
# processed_imputed = processed_full[processed_full.columns[processed_full.isna().any().tolist()]]

# processed_imputed = processed_imputed.replace([np.inf, -np.inf], np.nan)

In [33]:
# from sklearn.impute import KNNImputer

# imputer = KNNImputer(n_neighbors=1)
# imputed = imputer.fit_transform(processed_imputed)

# df_knn = pd.DataFrame(imputed, columns=processed_imputed.columns.tolist())

# for i in processed_imputed.columns.tolist():
#     processed_imputed[i] = df_knn[i]

In [34]:
# print(processed_imputed.isna().any())

In [35]:
# processed_imputed.head()

In [36]:
# processed_imputed.tail()

In [37]:
# from sklearn.model_selection import train_test_split 
# X = processed_imputed.astype(float).to_numpy()
# train, trade = train_test_split(X, test_size=0.33, shuffle=False)
# train = pd.DataFrame(train)
# trade = pd.DataFrame(trade)
# print(train.shape)
# print(trade.shape)
# # train = data_split(processed_full, '2018-05-16','2020-05-16')
# # trade = data_split(processed_full, '2020-05-17','2021-02-10')

In [38]:
# processed_full.date.head()

In [39]:
# train[["date","tic"]] = processed_full[["date","tic"]].iloc[:len(train),:]
# train.columns = processed_full.columns
# train = train.sort_values(["date", "tic"], ignore_index=True)
# train.index = train.date.factorize()[0]
# trade[["date","tic"]] = processed_full[["date","tic"]].iloc[:len(trade),:]
# trade.columns = processed_full.columns
# trade = trade.sort_values(["date", "tic"], ignore_index=True)
# trade.index = trade.date.factorize()[0]

In [40]:
train.head()

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma
0,2002-01-02,AMD,16.280001,16.410000,15.950,16.389999,4475400.0,2.0,0.000000,22.094359,13.665642,100.0,66.666667,100.0,16.389999,16.389999
0,2002-01-02,AMZN,10.930000,11.000000,10.480,10.960000,6519600.0,2.0,0.066859,22.094359,13.665642,100.0,66.666667,100.0,17.880000,17.880000
0,2002-01-02,APPN,0.000000,0.000000,0.000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000
0,2002-01-02,BABA,0.000000,0.000000,0.000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000
0,2002-01-02,BAC,31.155001,31.549999,31.055,19.802853,8669200.0,2.0,0.104728,22.443212,14.730121,100.0,80.580876,100.0,18.586667,18.586667


In [41]:
trade.head()

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma
0,2011-03-04,AMD,9.330000,9.39,9.150000,9.230000,23059400.0,4.0,-0.616934,38.323786,35.252215,40.648284,-214.107522,51.188546,37.269334,37.849333
0,2011-03-04,AMZN,172.619995,172.75,169.509995,171.669998,4924300.0,4.0,-0.832561,38.819371,34.324629,37.871162,-319.579205,59.111876,37.096000,37.705333
0,2011-03-04,APPN,0.000000,0.00,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
0,2011-03-04,BABA,0.000000,0.00,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
0,2011-03-04,BAC,14.300000,14.31,13.980000,12.403003,146184700.0,4.0,-0.864377,38.845690,34.058310,42.914434,-173.166650,21.243257,36.984000,37.592000


In [42]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 80815 entries, 0 to 2308
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   date          80815 non-null  object 
 1   tic           80815 non-null  object 
 2   open          80815 non-null  float64
 3   high          80815 non-null  float64
 4   low           80815 non-null  float64
 5   close         80815 non-null  float64
 6   volume        80815 non-null  float64
 7   day           80815 non-null  float64
 8   macd          80815 non-null  float64
 9   boll_ub       80815 non-null  float64
 10  boll_lb       80815 non-null  float64
 11  rsi_30        80815 non-null  float64
 12  cci_30        80815 non-null  float64
 13  dx_30         80815 non-null  float64
 14  close_30_sma  80815 non-null  float64
 15  close_60_sma  80815 non-null  float64
dtypes: float64(14), object(2)
memory usage: 10.5+ MB


In [43]:
len(train.tic.unique())

35

In [44]:
config["TECHNICAL_INDICATORS_LIST"]

['macd',
 'boll_ub',
 'boll_lb',
 'rsi_30',
 'cci_30',
 'dx_30',
 'close_30_sma',
 'close_60_sma']

In [45]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(config["TECHNICAL_INDICATORS_LIST"])*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

env_kwargs = {
    "hmax": 100, 
    "initial_amount": 1000000, 
    "buy_cost_pct": 0.001,
    "sell_cost_pct": 0.001,
    "state_space": state_space, 
    "stock_dim": stock_dimension, 
    "tech_indicator_list": config["TECHNICAL_INDICATORS_LIST"], 
    "action_space": stock_dimension, 
    "reward_scaling": 1e-4
    
}
e_train_gym = StockTradingEnv(df = train, **env_kwargs)

Stock Dimension: 35, State Space: 351


In [46]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


In [47]:
agent = DRLAgent(env = env_train)
PPO_PARAMS = {
    "n_steps": 2048,
    "ent_coef": 0.01,
    "learning_rate": 0.00025,
    "batch_size": 128,
}
model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)


{'n_steps': 2048, 'ent_coef': 0.01, 'learning_rate': 0.00025, 'batch_size': 128}
Using cpu device


In [48]:
trained_ppo = agent.train_model(model=model_ppo, 
                             tb_log_name='ppo',
                             total_timesteps=100000)

Logging to tensorboard_log/ppo/ppo_1
-----------------------------
| time/              |      |
|    fps             | 69   |
|    iterations      | 1    |
|    time_elapsed    | 29   |
|    total_timesteps | 2048 |
-----------------------------
----------------------------------------
| environment/            |            |
|    portfolio_value      | 2e+06      |
|    total_cost           | 9.14e+04   |
|    total_reward         | 1e+06      |
|    total_reward_pct     | 100        |
|    total_trades         | 43218      |
| time/                   |            |
|    fps                  | 74         |
|    iterations           | 2          |
|    time_elapsed         | 54         |
|    total_timesteps      | 4096       |
| train/                  |            |
|    approx_kl            | 0.01594159 |
|    clip_fraction        | 0.177      |
|    clip_range           | 0.2        |
|    entropy_loss         | -49.7      |
|    explained_variance   | -142       |
|    learning_r

In [49]:
e_trade_gym = StockTradingEnv(df = trade, **env_kwargs)
# env_trade, obs_trade = e_trade_gym.get_sb_env()

df_account_value, df_actions = DRLAgent.DRL_prediction(
    model=model_ppo, 
    environment = e_trade_gym)

hit end!


In [ ]:
df_account_value.shape

In [54]:
df_account_value.head()

,date,account_value
0,2011-03-04,1000000.000000
1,2011-03-07,998922.483783
2,2011-03-08,998821.804815
3,2011-03-09,998883.704559
4,2011-03-10,997188.392592


In [52]:
df_actions.head()

,AMD,AMZN,APPN,BABA,BAC,BB,BNGO,CCIV,EA,FUBO,GOOG,IBIO,IDEX,INTC,JMIA,JNJ,JPM,LUV,MA,MVIS,NDA,NIO,NOK,PFE,PG,PLTR,PYPL,RIOT,RLLCF,SNOW,TSLA,TSNP,UPS,USB,WFC
0,100,100,0,0,0,0,0,0,3,0,100,100,100,57,0,0,0,0,94,74,0,0,100,0,0,0,0,0,0,0,0,100,58,0,0
1,-93,-21,0,0,0,22,0,0,45,0,-100,-4,-100,-57,0,78,18,63,-47,100,38,0,-12,97,0,0,0,0,0,0,0,33,98,0,0
2,-7,81,0,0,0,-22,0,0,-48,0,52,43,0,0,0,-7,100,15,3,-36,100,0,64,-59,0,0,0,0,0,0,6,100,8,0,20
3,0,-14,0,0,0,60,0,0,100,0,100,57,100,14,0,25,-45,92,-50,86,54,0,61,-38,2,0,0,0,0,0,-6,59,45,37,24
4,100,55,0,0,100,-60,0,0,-3,0,100,-100,100,-14,0,-23,-73,-38,0,100,-27,0,-100,97,100,0,0,0,0,0,100,100,15,-21,76


In [53]:
print("==============Get Backtest Results===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value)
perf_stats_all = pd.DataFrame(perf_stats_all)
# perf_stats_all.to_csv("./"+config.RESULTS_DIR+"/perf_stats_all_"+now+'.csv')

==============Get Backtest Results===========
Annual return          0.200379
Cumulative returns     5.007643
Annual volatility      0.190000
Sharpe ratio           1.057203
Calmar ratio           0.668336
Stability              0.979071
Max drawdown          -0.299817
Omega ratio            1.218583
Sortino ratio          1.502597
Skew                        NaN
Kurtosis                    NaN
Tail ratio             0.967800
Daily value at risk   -0.023141
dtype: float64
==============Get Backtest Results===========
Annual return          0.200379
Cumulative returns     5.007643
Annual volatility      0.190000
Sharpe ratio           1.057203
Calmar ratio           0.668336
Stability              0.979071
Max drawdown          -0.299817
Omega ratio            1.218583
Sortino ratio          1.502597
Skew                        NaN
Kurtosis                    NaN
Tail ratio             0.967800
Daily value at risk   -0.023141
dtype: float64
